# Caps y Floors

Hasta ahora sólo hemos visto productos lineales de tasa de interés. Hoy introducimos el primer ejemplo de productos no lineales (u opciones) de tasa de interés.

## Configuración

In [1]:
from finrisk import QC_Financial_3 as Qcf
from IPython.core.display import HTML
from IPython.display import Image
import modules.auxiliary as aux
import pandas as pd

Se genera el calendario NY.

In [2]:
bus_cal = aux.get_cal(aux.BusCal.NY)

Esta función devuelve una pata flotante con Libor USD 3M.

In [27]:
def get_libor_usd_3m_leg(
        rp: Qcf.RecPay,
        notional: float,
        fecha_inicio: Qcf.QCDate,
        maturity: Qcf.Tenor,
        spread: float = 0.0) -> Qcf.Leg:

    # Se da de alta los parámetros requeridos
    periodicidad_pago = Qcf.Tenor('3M')
    periodo_irregular_pago = Qcf.StubPeriod.SHORTFRONT
    lag_pago = 0
    periodicidad_fijacion = Qcf.Tenor('3M')
    periodo_irregular_fijacion = Qcf.StubPeriod.SHORTFRONT
    bus_adj_rule = Qcf.BusyAdjRules.MODFOLLOW
    amort_es_flujo = True
    gearing = 1.0  # intereses -> gearing * Libor + spread
    
    # vamos a usar el mismo calendario para pago y fijaciones
    lag_de_fijacion = 2

    # Definición del índice
    codigo = 'LIBORUSD3M'
    lin_act360 = Qcf.QCInterestRate(0.0, Qcf.QCAct360(), Qcf.QCLinearWf())
    fixing_lag = Qcf.Tenor('2d')
    tenor = Qcf.Tenor('3m')
    usd = Qcf.QCUSD()
    libor_usd_3m = Qcf.InterestRateIndex(codigo,
                                         lin_act360,
                                         fixing_lag,
                                         tenor,
                                         bus_cal,
                                         bus_cal,
                                         usd)
    # Fin índice

    meses = maturity.get_years() * 12 + maturity.get_months()
    fecha_final = fecha_inicio.add_months(meses)
    ibor_leg = Qcf.LegFactory.build_bullet_ibor_leg(
        rp,
        fecha_inicio,
        fecha_final,
        bus_adj_rule,
        periodicidad_pago,
        periodo_irregular_pago,
        bus_cal,
        lag_pago,
        periodicidad_fijacion,
        periodo_irregular_fijacion,
        bus_cal,
        lag_de_fijacion,
        libor_usd_3m,
        notional,
        amort_es_flujo,
        usd,
        spread,
        gearing)

    return ibor_leg

Formato para la visualización de los `DataFrame`.

In [28]:
frmt = {
    'nominal': '{:,.2f}',
    'amortizacion': '{:,.2f}',
    'interes': '{:,.2f}',
    'flujo': '{:,.2f}',
    'interes_cap': '{:,.2f}',
    'interes_total': '{:,.2f}',
    'valor_tasa': '{:.6%}',
    'tasa': '{:.6%}'
}

## Black-Scholes-Merton Revisited

Recordemos la fórmula para una *call* sobre una acción que no paga dividendos.

$$
\begin{equation}
C\left(S,t\right)=e^{-rT}\left[Se^{rT}\cdot N\left(d_1\right)-K\cdot N\left(d_2\right)\right]
\end{equation}
$$

$$
\begin{equation}
d_1=\frac{\ln\frac{Se^{rT}}{K}+\frac{1}{2}\sigma^2T}{\sigma\sqrt{T}}
\end{equation}
$$

$$
\begin{equation}
d_2=d_1-\sigma\sqrt{T}
\end{equation}
$$

En la medida ajustada por riesgo tenemos que $Se^{rT}=\mathbb{E}_t^Q\left(S_T\right)$ y por lo tanto:

$$
\begin{equation}
C\left(S,t\right)=df\left(r,T\right)\left[\mathbb{E}_t^Q\left(S_T\right)\cdot N\left(d_1\right)-K\cdot N\left(d_2\right)\right]
\end{equation}
$$

$$
\begin{equation}
d_1=\frac{\ln\frac{\mathbb{E}_t^Q\left(S_T\right)}{K}+\frac{1}{2}\sigma^2T}{\sigma\sqrt{T}}
\end{equation}
$$

$$
\begin{equation}
d_2=d_1-\sigma\sqrt{T}
\end{equation}
$$

## Caps

- Un Cap es una opción que ofrece protección contra subidas de alguna tasa de interés.
- Por ejemplo, una empresa puede tener un financiamiento a tasa variable y estar interesada en tener protección contra incrementos de la tasa de referencia por sobre un cierto umbral.
- Veamos el ejemplo de un Cap al 0.35% para proteger un crédito en USD a 2Y a tasa flotante Libor 3M.

### Crédito a Libor USD 3M

Se construye un crédito a Libor USD 3M a 2Y.

In [29]:
credito = get_libor_usd_3m_leg(Qcf.RecPay.PAY, 10000000, Qcf.QCDate(26, 10, 2020), Qcf.Tenor('2Y'))

Visualizamos y vemos como ninguno de los cupones tiene su *fixing*, es decir el valor de la Libor a la fecha de fixing.

In [30]:
aux.show_leg(credito, 'IborCashflow', '').style.format(frmt)

,fecha_inicial,fecha_final,fecha_fixing,fecha_pago,nominal,amortizacion,interes,amort_es_flujo,flujo,moneda,codigo_indice_tasa,valor_tasa,spread,gearing,tipo_tasa
0,2020-10-26,2021-01-26,2020-10-22,2021-01-26,"-10,000,000.00",0.00,-0.00,True,0.00,USD,LIBORUSD3M,0.000000%,0.000000,1.000000,LinAct360
1,2021-01-26,2021-04-26,2021-01-22,2021-04-26,"-10,000,000.00",0.00,-0.00,True,0.00,USD,LIBORUSD3M,0.000000%,0.000000,1.000000,LinAct360
2,2021-04-26,2021-07-26,2021-04-22,2021-07-26,"-10,000,000.00",0.00,-0.00,True,0.00,USD,LIBORUSD3M,0.000000%,0.000000,1.000000,LinAct360
3,2021-07-26,2021-10-26,2021-07-22,2021-10-26,"-10,000,000.00",0.00,-0.00,True,0.00,USD,LIBORUSD3M,0.000000%,0.000000,1.000000,LinAct360
4,2021-10-26,2022-01-26,2021-10-22,2022-01-26,"-10,000,000.00",0.00,-0.00,True,0.00,USD,LIBORUSD3M,0.000000%,0.000000,1.000000,LinAct360
5,2022-01-26,2022-04-26,2022-01-24,2022-04-26,"-10,000,000.00",0.00,-0.00,True,0.00,USD,LIBORUSD3M,0.000000%,0.000000,1.000000,LinAct360
6,2022-04-26,2022-07-26,2022-04-22,2022-07-26,"-10,000,000.00",0.00,-0.00,True,0.00,USD,LIBORUSD3M,0.000000%,0.000000,1.000000,LinAct360
7,2022-07-26,2022-10-26,2022-07-22,2022-10-26,"-10,000,000.00","-10,000,000.00",-0.00,True,"-10,000,000.00",USD,LIBORUSD3M,0.000000%,0.000000,1.000000,LinAct360


[www.global-rates.com (valores Libor USD)](https://www.global-rates.com/en/interest-rates/libor/american-dollar/american-dollar.aspx)

Vamos a considerar un **escenario arbitrario** en que los *fixings* de la Libor aumentan 0.04% en cada nuevo cupón. El *fixing* del primer cupón corresponde al valor de la Libor USD 3M del 22-10-2020.

In [31]:
first_fixing = .0021475
increment = .0004 # Suponemos que los fixings sucesivos van aumentando en este monto.
for i in range(credito.size()):
    cshflw = credito.get_cashflow_at(i)
    cshflw.set_interest_rate_value(first_fixing + i * increment)

Vemos como, en este escenario, los intereses a pagar aumentan hasta más del doble del monto de intereses del primer cupón.

In [32]:
df_cred = aux.show_leg(credito, 'IborCashflow', '')
df_cred.style.format(frmt)

,fecha_inicial,fecha_final,fecha_fixing,fecha_pago,nominal,amortizacion,interes,amort_es_flujo,flujo,moneda,codigo_indice_tasa,valor_tasa,spread,gearing,tipo_tasa
0,2020-10-26,2021-01-26,2020-10-22,2021-01-26,"-10,000,000.00",0.00,"-5,488.06",True,"-5,488.06",USD,LIBORUSD3M,0.214750%,0.000000,1.000000,LinAct360
1,2021-01-26,2021-04-26,2021-01-22,2021-04-26,"-10,000,000.00",0.00,"-6,368.75",True,"-6,368.75",USD,LIBORUSD3M,0.254750%,0.000000,1.000000,LinAct360
2,2021-04-26,2021-07-26,2021-04-22,2021-07-26,"-10,000,000.00",0.00,"-7,450.63",True,"-7,450.63",USD,LIBORUSD3M,0.294750%,0.000000,1.000000,LinAct360
3,2021-07-26,2021-10-26,2021-07-22,2021-10-26,"-10,000,000.00",0.00,"-8,554.72",True,"-8,554.72",USD,LIBORUSD3M,0.334750%,0.000000,1.000000,LinAct360
4,2021-10-26,2022-01-26,2021-10-22,2022-01-26,"-10,000,000.00",0.00,"-9,576.94",True,"-9,576.94",USD,LIBORUSD3M,0.374750%,0.000000,1.000000,LinAct360
5,2022-01-26,2022-04-26,2022-01-24,2022-04-26,"-10,000,000.00",0.00,"-10,368.75",True,"-10,368.75",USD,LIBORUSD3M,0.414750%,0.000000,1.000000,LinAct360
6,2022-04-26,2022-07-26,2022-04-22,2022-07-26,"-10,000,000.00",0.00,"-11,495.07",True,"-11,495.07",USD,LIBORUSD3M,0.454750%,0.000000,1.000000,LinAct360
7,2022-07-26,2022-10-26,2022-07-22,2022-10-26,"-10,000,000.00","-10,000,000.00","-12,643.61",True,"-10,012,643.61",USD,LIBORUSD3M,0.494750%,0.000000,1.000000,LinAct360


### Crédito a Libor USD 3M + Cap

En un escenario como el anterior, quien ha suscrito el crédito, podría tener interés en contratar protección contra las subidas de la Libor. En particular, podría contratar un Cap de Libor a 2Y.

El payoff de un Cap, está dado por la siguiente función `caplet` asociada a cada uno de los cupones del crédito.

**NB:** la función se llama `caplet` por que corresponde a una componente del Cap total.

In [33]:
def caplet(notional: float,
        fixing: float,
        strike: float,
        fecha_inicio: Qcf.QCDate,
        fecha_final: Qcf.QCDate) -> float:
    
    valor_tasa = max(fixing - strike, 0)
    lin_act360 = Qcf.QCInterestRate(valor_tasa, Qcf.QCAct360(), Qcf.QCLinearWf())
    wf = lin_act360.wf(fecha_inicio, fecha_final)
    
    # Los interes que se devengan con valor_tasa entre la fecha_inicio y la fecha_final
    return notional * (wf - 1)

Apliquemos `caplet` a cada uno de los cupones y *fixings* del escenario, considerando una tasa Cap (o strike) de 0.35%.

In [34]:
strike = .0035
df_cred['interes_cap'] = df_cred.apply(
    lambda row: -caplet(
        row['nominal'],
        row['valor_tasa'],
        strike,
        Qcf.build_qcdate_from_string(row['fecha_inicial']),
        Qcf.build_qcdate_from_string(row['fecha_final'])
    ),
    axis=1
)

df_cred['interes_total'] = df_cred['interes'] + df_cred['interes_cap']

Vemos como cada `caplet` provee un flujo de caja positivo, cada vez que el *fixing* de la Libor supera el valor de la tasa Cap o strike del Cap.

Los últimos pagos de intereses (total) están a una tasa efectiva igual al *strike* (0.35%).

In [35]:
df_cred.style.format(frmt)

,fecha_inicial,fecha_final,fecha_fixing,fecha_pago,nominal,amortizacion,interes,amort_es_flujo,flujo,moneda,codigo_indice_tasa,valor_tasa,spread,gearing,tipo_tasa,interes_cap,interes_total
0,2020-10-26,2021-01-26,2020-10-22,2021-01-26,"-10,000,000.00",0.00,"-5,488.06",True,"-5,488.06",USD,LIBORUSD3M,0.214750%,0.000000,1.000000,LinAct360,0.00,"-5,488.06"
1,2021-01-26,2021-04-26,2021-01-22,2021-04-26,"-10,000,000.00",0.00,"-6,368.75",True,"-6,368.75",USD,LIBORUSD3M,0.254750%,0.000000,1.000000,LinAct360,0.00,"-6,368.75"
2,2021-04-26,2021-07-26,2021-04-22,2021-07-26,"-10,000,000.00",0.00,"-7,450.63",True,"-7,450.63",USD,LIBORUSD3M,0.294750%,0.000000,1.000000,LinAct360,0.00,"-7,450.63"
3,2021-07-26,2021-10-26,2021-07-22,2021-10-26,"-10,000,000.00",0.00,"-8,554.72",True,"-8,554.72",USD,LIBORUSD3M,0.334750%,0.000000,1.000000,LinAct360,0.00,"-8,554.72"
4,2021-10-26,2022-01-26,2021-10-22,2022-01-26,"-10,000,000.00",0.00,"-9,576.94",True,"-9,576.94",USD,LIBORUSD3M,0.374750%,0.000000,1.000000,LinAct360,632.50,"-8,944.44"
5,2022-01-26,2022-04-26,2022-01-24,2022-04-26,"-10,000,000.00",0.00,"-10,368.75",True,"-10,368.75",USD,LIBORUSD3M,0.414750%,0.000000,1.000000,LinAct360,"1,618.75","-8,750.00"
6,2022-04-26,2022-07-26,2022-04-22,2022-07-26,"-10,000,000.00",0.00,"-11,495.07",True,"-11,495.07",USD,LIBORUSD3M,0.454750%,0.000000,1.000000,LinAct360,"2,647.85","-8,847.22"
7,2022-07-26,2022-10-26,2022-07-22,2022-10-26,"-10,000,000.00","-10,000,000.00","-12,643.61",True,"-10,012,643.61",USD,LIBORUSD3M,0.494750%,0.000000,1.000000,LinAct360,"3,699.17","-8,944.44"


### Fórmula de Valorización

- Cada componente individual de un Cap se dice caplet
- Evaluemos un caplet: su pago está dado por

$$
\begin{equation}
yf\cdot N\cdot max\left(L_{T_f}-L_K,0\right)
\end{equation}
$$

- Donde $yf$ es la fracción de año que se usa para la tasa de referencia (lo más usual es Act/360), $N$ es el nocional del contrato, $L_K$ es la tasa Cap y $T_f$ es la fecha en la que se produce el *fixing*.
- Notar que, aunque el pago queda determinado al tiempo $T_f$, éste se realiza sólo cuando ha concluido el período de devengo dado por $yf$.

- Vemos, por lo tanto, que el pago de cada caplet está dado por una fórmula de Call sobre $L_{T_f}$ con strike $L_K$.
- Queremos entonces poder aplicar una fórmula de BSM para el valor del caplet.
- El mercado ha adoptado como estándar el uso de la fórmula de BSM para este producto (en el formato fácil de recordar que es utilizando el valor esperado del activo subyacente).

De esta forma se obtiene que:

$$
\begin{equation}
caplet=yf\cdot N\cdot e^{-r_{OIS}\cdot T_v}\left[\mathbb{E}_t^Q\left(L_{T_f}\right)\cdot N\left(d_1\right)-L_K\cdot N\left(d_2\right)\right]
\end{equation}
$$

$$
\begin{equation}
d_1=\frac{\ln\frac{\mathbb{E}_t^Q\left(L_{T_f}\right)}{L_K}+\frac{1}{2}\sigma^2T_f}{\sigma\sqrt{T_f}}
\end{equation}
$$

$$
\begin{equation}
d_2=d_1-\sigma\sqrt{T_f}
\end{equation}
$$

Donde $T_f$ es el plazo hasta el fixing de la opción, $T_v$ es el plazo hasta el pago de la opción y la tasa de descuento es la que proviene de la curva *OIS*.

## Floors

- Un Floor es una opción que ofrece protección contra bajadas de alguna tasa de interés.
- Por ejemplo, un inversionista puede tener activos a tasa variable y estar interesado en protección contra disminuciones de la tasa de referencia por debajo un cierto umbral.
- Veamos el ejemplo de un Floor al 0.01% para proteger un activo en USD a 2Y a tasa flotante Libor 3M.

### Activo a Libor USD 3M

Se construye un activo al Libor USD 3M. Podría ser un bono o el mismo crédito anterior desde el punto de vista del banco que lo otorga.

In [36]:
activo = get_libor_usd_3m_leg(Qcf.RecPay.RECEIVE,
                              10000000, Qcf.QCDate(26, 10, 2020), Qcf.Tenor('2Y'))

Visualizamos y vemos como ninguno de los cupones tiene su *fixing*, es decir el valor de la Libor a la fecha de fixing.

In [37]:
aux.show_leg(activo, 'IborCashflow', '').style.format(frmt)

,fecha_inicial,fecha_final,fecha_fixing,fecha_pago,nominal,amortizacion,interes,amort_es_flujo,flujo,moneda,codigo_indice_tasa,valor_tasa,spread,gearing,tipo_tasa
0,2020-10-26,2021-01-26,2020-10-22,2021-01-26,"10,000,000.00",0.00,0.00,True,0.00,USD,LIBORUSD3M,0.000000%,0.000000,1.000000,LinAct360
1,2021-01-26,2021-04-26,2021-01-22,2021-04-26,"10,000,000.00",0.00,0.00,True,0.00,USD,LIBORUSD3M,0.000000%,0.000000,1.000000,LinAct360
2,2021-04-26,2021-07-26,2021-04-22,2021-07-26,"10,000,000.00",0.00,0.00,True,0.00,USD,LIBORUSD3M,0.000000%,0.000000,1.000000,LinAct360
3,2021-07-26,2021-10-26,2021-07-22,2021-10-26,"10,000,000.00",0.00,0.00,True,0.00,USD,LIBORUSD3M,0.000000%,0.000000,1.000000,LinAct360
4,2021-10-26,2022-01-26,2021-10-22,2022-01-26,"10,000,000.00",0.00,0.00,True,0.00,USD,LIBORUSD3M,0.000000%,0.000000,1.000000,LinAct360
5,2022-01-26,2022-04-26,2022-01-24,2022-04-26,"10,000,000.00",0.00,0.00,True,0.00,USD,LIBORUSD3M,0.000000%,0.000000,1.000000,LinAct360
6,2022-04-26,2022-07-26,2022-04-22,2022-07-26,"10,000,000.00",0.00,0.00,True,0.00,USD,LIBORUSD3M,0.000000%,0.000000,1.000000,LinAct360
7,2022-07-26,2022-10-26,2022-07-22,2022-10-26,"10,000,000.00","10,000,000.00",0.00,True,"10,000,000.00",USD,LIBORUSD3M,0.000000%,0.000000,1.000000,LinAct360


Vamos a considerar un **escenario arbitrario** en que los *fixings* de la Libor disminuyen 0.03% en cada nuevo cupón. El *fixing* del primer cupón corresponde al valor de la Libor USD 3M del 22-10-2020.

In [38]:
increment = -.0003 # Suponemos que los fixings sucesivos van cambiando en este monto.
for i in range(activo.size()):
    cshflw = activo.get_cashflow_at(i)
    cshflw.set_interest_rate_value(first_fixing + i * increment)

Vemos como, en este escenario, los intereses por recibir llegan casi a 0 en el último cupón.

In [39]:
df_activo = aux.show_leg(activo, 'IborCashflow', '')
df_activo.style.format(frmt)

,fecha_inicial,fecha_final,fecha_fixing,fecha_pago,nominal,amortizacion,interes,amort_es_flujo,flujo,moneda,codigo_indice_tasa,valor_tasa,spread,gearing,tipo_tasa
0,2020-10-26,2021-01-26,2020-10-22,2021-01-26,"10,000,000.00",0.00,"5,488.06",True,"5,488.06",USD,LIBORUSD3M,0.214750%,0.000000,1.000000,LinAct360
1,2021-01-26,2021-04-26,2021-01-22,2021-04-26,"10,000,000.00",0.00,"4,618.75",True,"4,618.75",USD,LIBORUSD3M,0.184750%,0.000000,1.000000,LinAct360
2,2021-04-26,2021-07-26,2021-04-22,2021-07-26,"10,000,000.00",0.00,"3,911.74",True,"3,911.74",USD,LIBORUSD3M,0.154750%,0.000000,1.000000,LinAct360
3,2021-07-26,2021-10-26,2021-07-22,2021-10-26,"10,000,000.00",0.00,"3,188.06",True,"3,188.06",USD,LIBORUSD3M,0.124750%,0.000000,1.000000,LinAct360
4,2021-10-26,2022-01-26,2021-10-22,2022-01-26,"10,000,000.00",0.00,"2,421.39",True,"2,421.39",USD,LIBORUSD3M,0.094750%,0.000000,1.000000,LinAct360
5,2022-01-26,2022-04-26,2022-01-24,2022-04-26,"10,000,000.00",0.00,"1,618.75",True,"1,618.75",USD,LIBORUSD3M,0.064750%,0.000000,1.000000,LinAct360
6,2022-04-26,2022-07-26,2022-04-22,2022-07-26,"10,000,000.00",0.00,878.40,True,878.40,USD,LIBORUSD3M,0.034750%,0.000000,1.000000,LinAct360
7,2022-07-26,2022-10-26,2022-07-22,2022-10-26,"10,000,000.00","10,000,000.00",121.39,True,"10,000,121.39",USD,LIBORUSD3M,0.004750%,0.000000,1.000000,LinAct360


### Activo a Libor USD 3M + Floor

En un escenario como el anterior, el tenedor del bono o crédito, podría tener interés en contratar protección contra las bajadas de la Libor. En particular, podría contratar un Floor de Libor a 2Y.

El payoff de un Floor, está dado por la siguiente función `floorlet` asociada a cada uno de los cupones del activo.

**NB:** la función se llama `floorlet` por que corresponde a una componente del Floor total.

In [40]:
def floorlet(notional: float,
             fixing: float,
             strike: float,
             fecha_inicio: Qcf.QCDate,
             fecha_final: Qcf.QCDate) -> float:
    
    valor_tasa = max(strike - fixing, 0)
    lin_act360 = Qcf.QCInterestRate(valor_tasa, Qcf.QCAct360(), Qcf.QCLinearWf())
    wf = lin_act360.wf(fecha_inicio, fecha_final)
    return notional * (wf - 1)

Apliquemos `floorlet` a cada uno de los cupones y *fixings* del escenario, considerando una tasa Floor (o strike) de 0.15%.

In [41]:
strike = .0015
df_activo['interes_floor'] = df_activo.apply(
    lambda row: floorlet(
        row['nominal'],
        row['valor_tasa'],
        strike,
        Qcf.build_qcdate_from_string(row['fecha_inicial']),
        Qcf.build_qcdate_from_string(row['fecha_final'])
    ),
    axis=1
)

df_activo['interes_total'] = df_activo['interes'] + df_activo['interes_floor']

Vemos como cada `floorlet` provee un flujo de caja positivo, cada vez que el *fixing* de la Libor es inferior al valor de la tasa Floor o strike del Floor.

Los últimos pagos de intereses (total) están a una tasa efectiva igual al *strike* (0.15%).

In [42]:
df_activo.style.format(frmt)

,fecha_inicial,fecha_final,fecha_fixing,fecha_pago,nominal,amortizacion,interes,amort_es_flujo,flujo,moneda,codigo_indice_tasa,valor_tasa,spread,gearing,tipo_tasa,interes_floor,interes_total
0,2020-10-26,2021-01-26,2020-10-22,2021-01-26,"10,000,000.00",0.00,"5,488.06",True,"5,488.06",USD,LIBORUSD3M,0.214750%,0.000000,1.000000,LinAct360,0.000000,"5,488.06"
1,2021-01-26,2021-04-26,2021-01-22,2021-04-26,"10,000,000.00",0.00,"4,618.75",True,"4,618.75",USD,LIBORUSD3M,0.184750%,0.000000,1.000000,LinAct360,0.000000,"4,618.75"
2,2021-04-26,2021-07-26,2021-04-22,2021-07-26,"10,000,000.00",0.00,"3,911.74",True,"3,911.74",USD,LIBORUSD3M,0.154750%,0.000000,1.000000,LinAct360,0.000000,"3,911.74"
3,2021-07-26,2021-10-26,2021-07-22,2021-10-26,"10,000,000.00",0.00,"3,188.06",True,"3,188.06",USD,LIBORUSD3M,0.124750%,0.000000,1.000000,LinAct360,645.277778,"3,833.33"
4,2021-10-26,2022-01-26,2021-10-22,2022-01-26,"10,000,000.00",0.00,"2,421.39",True,"2,421.39",USD,LIBORUSD3M,0.094750%,0.000000,1.000000,LinAct360,1411.944444,"3,833.33"
5,2022-01-26,2022-04-26,2022-01-24,2022-04-26,"10,000,000.00",0.00,"1,618.75",True,"1,618.75",USD,LIBORUSD3M,0.064750%,0.000000,1.000000,LinAct360,2131.250000,"3,750.00"
6,2022-04-26,2022-07-26,2022-04-22,2022-07-26,"10,000,000.00",0.00,878.40,True,878.40,USD,LIBORUSD3M,0.034750%,0.000000,1.000000,LinAct360,2913.263889,"3,791.67"
7,2022-07-26,2022-10-26,2022-07-22,2022-10-26,"10,000,000.00","10,000,000.00",121.39,True,"10,000,121.39",USD,LIBORUSD3M,0.004750%,0.000000,1.000000,LinAct360,3711.944444,"3,833.33"


### Fórmula de Valorización

- Cada componente individual de un floor se dice floorlet.
- Evaluemos un floorlet: su pago está dado por

$$
\begin{equation}
yf\cdot N\cdot max\left(L_K-L_{T_f},0\right)
\end{equation}
$$

- Donde $yf$ es la fracción de año que se usa para la tasa de referencia (lo más usual es Act/360), $N$ es el nocional del contrato y $L_K$ es la tasa floor.
- Notar que, aunque el pago queda determinado al tiempo $T_f$, éste se realiza sólo cuando ha concluido el período de devengo dado por $yf$.

- Vemos, por lo tanto, que el pago de cada floorlet está dado por una fórmula de Put sobre $L_{T_f}$ con strike $L_K$
- Queremos entonces poder aplicar una fórmula de BSM para el valor del floorlet.
- El mercado ha adoptado como estándar el uso de la fórmula de BSM para este producto (en el formato fácil de recordar que es utilizando el valor esperado del activo subyacente).

De esta forma se obtiene que:

$$
\begin{equation}
floorlet=yf\cdot N\cdot e^{-r_{OIS}\cdot T_v}\left[L_K\cdot N\left(-d_2\right)-\mathbb{E}_t^Q\left(L_{T_f}\right)\cdot N\left(-d_1\right)\right]
\end{equation}
$$

$$
\begin{equation}
d_1=\frac{\ln\frac{\mathbb{E}_t^Q\left(L_{T_f}\right)}{L_K}+\frac{1}{2}\sigma^2T_f}{\sigma\sqrt{T_f}}
\end{equation}
$$

$$
\begin{equation}
d_2=d_1-\sigma\sqrt{T_f}
\end{equation}
$$

Donde $T_f$ es el plazo hasta el fixing de la opción, $T_v$ es el plazo hasta el pago de la opción y la tasa de descuento es la que proviene de la curva *OIS*.

## Ejercicio

Recordemos la Put – Call parity, ésta nos dice que:

$$
\begin{equation}
Call-Put=Forward
\end{equation}
$$

- ¿Qué forma asume esta relación para el caso de Caps y Floors?
- Notar que si compro un Cap y un Floor con la misma estructura temporal y el mismo strike sucede que:
  - Si la tasa flotante está por arriba del strike recibo este exceso
  - Si la tasa flotante está por debajo del strike pago este déficit

$$
\begin{equation}
Cap-Floor=Swap
\end{equation}
$$

## Matriz de Volatilidad

En esta pantalla para cotizar Caps, se puede ver el valor del parámetro de volatilidad utilizado para calcular el valor del Cap (campo NPV).

In [20]:
Image(url="img/20201027_cap.gif", width=900, height=720)

Dicha volatilidad se deduce de la siguiente matriz de volatilidad, la cual, a su vez, proviene de cotizaciones de bancos y otras instituciones activas en este mercado.

In [21]:
Image(url="img/20201027_cap_vol.gif", width=900, height=720)

In [22]:
curvas = pd.read_csv('data/20201027_curvas.csv')
curvas.columns = ['fecha', 'plazo', 'tasa', 'codigo']

In [23]:
curvas.style.format(frmt)

,fecha,plazo,tasa,codigo
0,2020-10-27,1,-0.473373%,EURIBOR6M
1,2020-10-27,4,-0.509682%,EURIBOR6M
2,2020-10-27,7,-0.545975%,EURIBOR6M
3,2020-10-27,14,-0.545947%,EURIBOR6M
4,2020-10-27,32,-0.549275%,EURIBOR6M
5,2020-10-27,61,-0.533269%,EURIBOR6M
6,2020-10-27,92,-0.516071%,EURIBOR6M
7,2020-10-27,120,-0.515312%,EURIBOR6M
8,2020-10-27,151,-0.514439%,EURIBOR6M
9,2020-10-27,182,-0.513428%,EURIBOR6M


In [43]:
df_libor3m = curvas[curvas.codigo == 'LIBORUSD3MBBG']
df_sofr = curvas[curvas.codigo == 'USDSOFR']

In [44]:
df_sofr.style.format(frmt)

,fecha,plazo,tasa,codigo
112,2020-10-27,1,0.073027%,USDSOFR
113,2020-10-27,4,0.082159%,USDSOFR
114,2020-10-27,7,0.083464%,USDSOFR
115,2020-10-27,14,0.084769%,USDSOFR
116,2020-10-27,32,0.085587%,USDSOFR
117,2020-10-27,61,0.082913%,USDSOFR
118,2020-10-27,92,0.083954%,USDSOFR
119,2020-10-27,120,0.081865%,USDSOFR
120,2020-10-27,151,0.079934%,USDSOFR
121,2020-10-27,182,0.078561%,USDSOFR


## Ejercicio

Utilizando las curvas *Libor* y *SOFR* de más arriba, valorice el Cap a 5Y con strike ATM. Las convenciones de tasas de ambas curvas es Com Act/365.

**NOTA:** El valor esperado de la Libor a una cierta fecha usando la curva **Libor de BBG** se calcula como:

$$
\begin{equation}
\mathbb{E}_t^Q\left(L_{T_f}\right)=\left(\frac{P_L\left(t,T_f\right)}{P_L\left(t,T_{f+3M}\right)}-1\right)\cdot\frac{1}{yf\left(T_f,T_{f+3M}\right)}
\end{equation}
$$

Donde $P_L\left(t,T\right)$ es el factor de descuento obtenido desde la curva Libor entre $t=hoy$ y $T$ y $f$ es la fecha de inicio de la Libor.

Idealmente, utilicen funciones de la librería.

In [45]:
plazos = [90, 180, 270, 360]
tasas = [.01, .012, .014, .016]

In [48]:
df_90 = 1/(1+tasas[0])**(plazos[0]/365)
df_90

0.9975495011486121

In [49]:
df_180 = 1/(1+tasas[1])**(plazos[1]/365)
df_180

0.9941346856205336

In [52]:
ValorEsperado_L3M_en_90D = (df_90 / df_180 - 1) * 360 / (plazos[1] - plazos[0])
ValorEsperado_L3M_en_90D

0.013739850655937857